# AutoCropper for letters associated with Loudon Bridge

1. Letters were shot on a Sony A7r2 with Voigtlander 65mm APO-Macro lens tethered to Capture One Pro 12 software
1. Letters were digitized with a white background
1. Measured resolution for files is 410ppi
1. Images exported as 8-bit \*.tif files in the AdobeRGB(1998) colorspace


In [1]:
# importing
from pathlib import Path
from subprocess import call

import cv2
import numpy as np
from PIL import Image

import img_qc.img_qc as img_qc

# == matplotlib options

# magic that lets us plot directly in the notebook
%matplotlib inline
import matplotlib.pyplot as plt

# parameters for matplotlib to increase our default figure size -- NOTE: figure sizes are in INCHES
plt.rcParams["figure.figsize"] = (20,20)  # set as needed for your screen and eyes

In [41]:
# functions

def rotate_bound(image, angle, center=None, scale=1.0):
    # grab the dimensions of the image and then determine the
    # center
    (height, width) = image.shape[:2]
    
    # if the center is None, initialize it as the center of
    # the image
    if center is None:
        centerX = (width // 2)
        centerY = (height // 2)
    else:
        centerX, centerY = center

    # grab the rotation matrix (applying the negative of the
    # angle to rotate clockwise), then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((centerX, centerY), -angle, scale)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])

    # compute the new bounding dimensions of the image
    width_new = int((height * sin) + (width * cos))
    height_new = int((height * cos) + (width * sin))

    # adjust the rotation matrix to take into account translation
    M[0, 2] += (width_new / 2) - centerX
    M[1, 2] += (height_new / 2) - centerY

    # perform the actual rotation and return the image
    return cv2.warpAffine(image, M, (width_new, height_new), flags=cv2.INTER_CUBIC)

def autocrop(image_path, compression=None, resize_height=500, dpi=400, padding=0, original_icc_profile=False, rotate_odd_even=False):
    
    # === AutoCrop

    # load the image
    image = cv2.imread(str(image_path))
                       
    # crop image inside blue tape, but don't use if not necessary
    #image = image[1200:8600, 400:5000]
    
    # rotate image CCW for odd numbered and CW for even numbered images
    if rotate_odd_even:
        if (int(image_path.stem[-1]) % 2 == 0):
            image = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
        else:
            image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
    
    # compute the ratio of the old height to the new height
    ratio = image.shape[0] / float(resize_height)
    
    # clone image
    image_original = image.copy()
    
    # resize image
    image = img_qc.get_resized_cv_image(image, height=resize_height)
    
    # convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # blur the image
    blurred = cv2.GaussianBlur(gray, (9, 9), 0)
    
    # apply Otsu's automatic thresholding
    (T, thresh) = cv2.threshold(blurred, 0, 255, 
                               cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
    
    # DEBUG: show thresholded image
    #clone = thresh.copy()
    #plt.imshow(clone, cmap='gray')
    
    # find the contours in the thresholded image keeping the external one
    contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # sort the contours from left to right
    (cnts, bounding_boxes) = img_qc.sort_contours(contours)
    
    # DEBUG
    # clone = image.copy()
    
    # loop over the contours individually
    for (i, c) in enumerate(cnts):
        # if the contour is not sufficiently large, ignore it
        if cv2.contourArea(c) < 20000:  # use 20000 for scrapbook pages
            continue
            
        # compute the rotated bounding box of the contour
        box = cv2.minAreaRect(c)
        box = cv2.boxPoints(box)
        box = np.int0(box)
        
        # DEBUG: draw found contour & show image
        #clone = cv2.cvtColor(clone, cv2.COLOR_BGR2RGB)
        #cv2.drawContours(clone, [box], 0, (0, 0, 255), 2)
        #plt.imshow(clone)
        
        # re-order the points in tl, tr, br, bl order
        rect = img_qc.order_points(box)
        
        # find the points and angle for minAreaRectangle
        (x, y), (w, h), theta = cv2.minAreaRect(c)
        
        # rotate image around center of minAreaRect by theta amount
        if theta < -45:
            theta = 90 + theta
        
        # DEBUG: rotate & show image
        #clone = image.copy()
        #clone = cv2.cvtColor(clone, cv2.COLOR_BGR2RGB)
        #image_rotated = img_qc.rotate(clone, theta, (x, y))
        #plt.imshow(image_rotated)
    
    # multiply the rectangle by the original ratio
    rect *= ratio
    
    # find the points we need to crop the full size original
    tl, tr, br, bl = rect
    startX = max(min(tl[0], bl[0]), 0)
    startY = max(min(tl[1], tr[1]), 0)
    endX = max(tr[0], br[0])
    endY = max(bl[1], br[1])
    
    # rotate original by theta from minAreaRect
    x *= ratio
    y *= ratio
    image_rotated = img_qc.rotate(image_original, theta, (x, y))
    
    # add padding (default hard-coded is 0 pixels)
    pixel_padding = int(padding)
    startX -= pixel_padding
    startY -= pixel_padding
    endX += pixel_padding
    endY += pixel_padding
    
    # startX/startY to max of current value and 0 to stay inside image
    startX = max(startX, 0)
    startY = max(startY, 0)
    
    # endX/endY to min of current value and max width/height of image to stay inside image
    endX = min(endX, image_rotated.shape[1])
    endY = min(endY, image_rotated.shape[0])
    
    # crop the image in memory
    image_cropped = image_rotated[int(startY):int(endY), int(startX):int(endX)]
    
    # create output directory and set output path
    output_directory_path = image_path.parents[0].joinpath('00_cropped')
    output_directory_path.mkdir(exist_ok=True)
    output_path = output_directory_path.joinpath(image_path.name)
    
    # convert to pillow Image
    image_cropped = cv2.cvtColor(image_cropped, cv2.COLOR_BGR2RGB)  # convert to RGB!
    pillow_image = Image.fromarray(image_cropped)
    
    dpi = float(dpi)  # dpi MUST be a float for Pillow
        
    pillow_image.save(output_path, compression=compression, dpi=(dpi, dpi))
    
    if original_icc_profile:
        # use exiftool to copy ICC profile form original to output image
        cmd = f' exiftool -tagsfromfile {str(image_path)} -ICC_Profile -overwrite_original -q {str(output_path)}'
        call(cmd, shell=True)

In [3]:
%%time
autocrop_directory_path = Path('data/ex1/')
autocrop_directory_path

Wall time: 1 ms


In [4]:
autocrop_directory_path.resolve()

WindowsPath('C:/Users/dlisla/Documents/GitHub/utk_LoudonBridge/data/ex1')

In [5]:
image_paths_list = sorted(autocrop_directory_path.glob('*.tif'))
image_paths_list

[WindowsPath('data/ex1/MS2685-1-1-1_0001.tif'),
 WindowsPath('data/ex1/MS2685-1-1-1_0002.tif'),
 WindowsPath('data/ex1/MS2685-1-1-1_0003.tif'),
 WindowsPath('data/ex1/MS2685-1-1-1_0004.tif'),
 WindowsPath('data/ex1/MS2685-1-1-1_0005.tif'),
 WindowsPath('data/ex1/MS2685-1-1-1_0006.tif'),
 WindowsPath('data/ex1/MS2685-1-1-1_0007.tif'),
 WindowsPath('data/ex1/MS2685-1-1-1_0008.tif'),
 WindowsPath('data/ex1/MS2685-1-1-1_0009.tif'),
 WindowsPath('data/ex1/MS2685-1-1-1_0010.tif'),
 WindowsPath('data/ex1/MS2685-1-1-1_0011.tif'),
 WindowsPath('data/ex1/MS2685-1-1-1_0012.tif'),
 WindowsPath('data/ex1/MS2685-1-1-1_0013.tif'),
 WindowsPath('data/ex1/MS2685-1-1-1_0014.tif'),
 WindowsPath('data/ex1/MS2685-1-1-1_0015.tif'),
 WindowsPath('data/ex1/MS2685-1-1-1_0016.tif'),
 WindowsPath('data/ex1/MS2685-1-1-1_0017.tif'),
 WindowsPath('data/ex1/MS2685-1-1-1_0018.tif'),
 WindowsPath('data/ex1/MS2685-1-1-1_0019.tif'),
 WindowsPath('data/ex1/MS2685-1-1-1_0020.tif'),
 WindowsPath('data/ex1/MS2685-1-1-1_0021

In [6]:
image_paths_list

[WindowsPath('data/ex1/MS2685-1-1-1_0001.tif'),
 WindowsPath('data/ex1/MS2685-1-1-1_0002.tif'),
 WindowsPath('data/ex1/MS2685-1-1-1_0003.tif'),
 WindowsPath('data/ex1/MS2685-1-1-1_0004.tif'),
 WindowsPath('data/ex1/MS2685-1-1-1_0005.tif'),
 WindowsPath('data/ex1/MS2685-1-1-1_0006.tif'),
 WindowsPath('data/ex1/MS2685-1-1-1_0007.tif'),
 WindowsPath('data/ex1/MS2685-1-1-1_0008.tif'),
 WindowsPath('data/ex1/MS2685-1-1-1_0009.tif'),
 WindowsPath('data/ex1/MS2685-1-1-1_0010.tif'),
 WindowsPath('data/ex1/MS2685-1-1-1_0011.tif'),
 WindowsPath('data/ex1/MS2685-1-1-1_0012.tif'),
 WindowsPath('data/ex1/MS2685-1-1-1_0013.tif'),
 WindowsPath('data/ex1/MS2685-1-1-1_0014.tif'),
 WindowsPath('data/ex1/MS2685-1-1-1_0015.tif'),
 WindowsPath('data/ex1/MS2685-1-1-1_0016.tif'),
 WindowsPath('data/ex1/MS2685-1-1-1_0017.tif'),
 WindowsPath('data/ex1/MS2685-1-1-1_0018.tif'),
 WindowsPath('data/ex1/MS2685-1-1-1_0019.tif'),
 WindowsPath('data/ex1/MS2685-1-1-1_0020.tif'),
 WindowsPath('data/ex1/MS2685-1-1-1_0021

In [7]:
# autocrop(image_paths_list[2], dpi=410, padding=0, original_icc_profile=True)

In [42]:
# process every image in image_paths_list
count = 0
for image_path in image_paths_list:
    print(f'starting to process {image_path.name}')
    autocrop(image_path, dpi=410, resize_height=500, padding=50, original_icc_profile=True)
    cropped_image_path = autocrop_directory_path.joinpath('00_cropped', image_path.name)
    if cropped_image_path.is_file():
        print(f'{image_path.name} was cropped')
    count += 1


starting to process MS2685-1-1-1_0001.tif
MS2685-1-1-1_0001.tif was cropped
starting to process MS2685-1-1-1_0002.tif
MS2685-1-1-1_0002.tif was cropped
starting to process MS2685-1-1-1_0003.tif
MS2685-1-1-1_0003.tif was cropped
starting to process MS2685-1-1-1_0004.tif
MS2685-1-1-1_0004.tif was cropped
starting to process MS2685-1-1-1_0005.tif
MS2685-1-1-1_0005.tif was cropped
starting to process MS2685-1-1-1_0006.tif
MS2685-1-1-1_0006.tif was cropped
starting to process MS2685-1-1-1_0007.tif
MS2685-1-1-1_0007.tif was cropped
starting to process MS2685-1-1-1_0008.tif
MS2685-1-1-1_0008.tif was cropped
starting to process MS2685-1-1-1_0009.tif
MS2685-1-1-1_0009.tif was cropped
starting to process MS2685-1-1-1_0010.tif
MS2685-1-1-1_0010.tif was cropped
starting to process MS2685-1-1-1_0011.tif
MS2685-1-1-1_0011.tif was cropped
starting to process MS2685-1-1-1_0012.tif
MS2685-1-1-1_0012.tif was cropped
starting to process MS2685-1-1-1_0013.tif
MS2685-1-1-1_0013.tif was cropped
starting to 

UnboundLocalError: local variable 'rect' referenced before assignment

In [43]:
# get count value to see the index of the last image that was trying to be cropped (or cropped if all successful)
count

40

In [44]:
# double-check that image_paths_list[count] is the correct value
image_paths_list[40]

WindowsPath('data/ex1/MS2685-1-1-1_0043.tif')

In [45]:
# identifies the last file that was counted
image_paths_list[count]

WindowsPath('data/ex1/MS2685-1-1-1_0043.tif')

In [51]:
# lists the files from the last counted files through the end of the list; slices list 
image_paths_list[count:count+2]

[WindowsPath('data/ex1/MS2685-1-1-1_0043.tif'),
 WindowsPath('data/ex1/MS2685-1-1-1_0044.tif')]

In [52]:
# process every image in image_paths_list
print(f'--- starting with image_paths_list[{count}]')

for image_path in image_paths_list[count:count+2]:
    print(f'starting to process {image_path.name}')
    autocrop(image_path, dpi=410, resize_height=1000, padding=50, original_icc_profile=True)
    cropped_image_path = autocrop_directory_path.joinpath('00_cropped', image_path.name)
    if cropped_image_path.is_file():
        print(f'{image_path.name} was cropped')
    count += 1



 starting with image_paths_list[40]
starting to process MS2685-1-1-1_0043.tif
MS2685-1-1-1_0043.tif was cropped
starting to process MS2685-1-1-1_0044.tif
MS2685-1-1-1_0044.tif was cropped


In [53]:
count

42

In [56]:
# process every image in image_paths_list, adding in a set number for count because i done goofed and needed to go
#back to that file
count=42
print(f'--- starting with image_paths_list[{count}]')
for image_path in image_paths_list[count:]:
    print(f'starting to process {image_path.name}')
    autocrop(image_path, dpi=410, resize_height=500, padding=50, original_icc_profile=True)
    cropped_image_path = autocrop_directory_path.joinpath('00_cropped', image_path.name)
    if cropped_image_path.is_file():
        print(f'{image_path.name} was cropped')
    count += 1


--- starting with image_paths_list[42]
starting to process MS2685-1-1-1_0045.tif
MS2685-1-1-1_0045.tif was cropped
starting to process MS2685-1-1-1_0046.tif
MS2685-1-1-1_0046.tif was cropped
starting to process MS2685-1-1-1_0047.tif
MS2685-1-1-1_0047.tif was cropped
starting to process MS2685-1-1-1_0048.tif
MS2685-1-1-1_0048.tif was cropped
starting to process MS2685-1-1-1_0049.tif
MS2685-1-1-1_0049.tif was cropped
starting to process MS2685-1-1-1_0050.tif
MS2685-1-1-1_0050.tif was cropped
starting to process MS2685-1-1-1_0051.tif
MS2685-1-1-1_0051.tif was cropped
starting to process MS2685-1-1-1_0052.tif
MS2685-1-1-1_0052.tif was cropped
starting to process MS2685-1-1-1_0053.tif
MS2685-1-1-1_0053.tif was cropped
starting to process MS2685-1-1-1_0054.tif
MS2685-1-1-1_0054.tif was cropped
starting to process MS2685-1-1-1_0055.tif


UnboundLocalError: local variable 'rect' referenced before assignment

In [57]:
count

52

In [58]:
image_paths_list[count]

WindowsPath('data/ex1/MS2685-1-1-1_0055.tif')

In [59]:
# process every image in image_paths_list
print(f'--- starting with image_paths_list[{count}]')

for image_path in image_paths_list[count:count+2]:
    print(f'starting to process {image_path.name}')
    autocrop(image_path, dpi=410, resize_height=1000, padding=50, original_icc_profile=True)
    cropped_image_path = autocrop_directory_path.joinpath('00_cropped', image_path.name)
    if cropped_image_path.is_file():
        print(f'{image_path.name} was cropped')
    count += 1


--- starting with image_paths_list[52]
starting to process MS2685-1-1-1_0055.tif
MS2685-1-1-1_0055.tif was cropped
starting to process MS2685-1-1-1_0056.tif
MS2685-1-1-1_0056.tif was cropped


In [60]:
# process every image in image_paths_list, adding in a set number for count because i done goofed and needed to go
#back to that file

print(f'--- starting with image_paths_list[{count}]')
for image_path in image_paths_list[count:]:
    print(f'starting to process {image_path.name}')
    autocrop(image_path, dpi=410, resize_height=500, padding=50, original_icc_profile=True)
    cropped_image_path = autocrop_directory_path.joinpath('00_cropped', image_path.name)
    if cropped_image_path.is_file():
        print(f'{image_path.name} was cropped')
    count += 1

--- starting with image_paths_list[54]
starting to process MS2685-1-1-1_0057.tif
MS2685-1-1-1_0057.tif was cropped
starting to process MS2685-1-1-1_0058.tif
MS2685-1-1-1_0058.tif was cropped
starting to process MS2685-1-1-1_0059.tif
MS2685-1-1-1_0059.tif was cropped
starting to process MS2685-1-1-1_0060.tif
MS2685-1-1-1_0060.tif was cropped
starting to process MS2685-1-1-1_0061.tif
MS2685-1-1-1_0061.tif was cropped
starting to process MS2685-1-1-1_0062.tif
MS2685-1-1-1_0062.tif was cropped
starting to process MS2685-1-1-1_0063.tif
MS2685-1-1-1_0063.tif was cropped
starting to process MS2685-1-1-1_0064.tif
MS2685-1-1-1_0064.tif was cropped
starting to process MS2685-1-1-1_0065.tif
MS2685-1-1-1_0065.tif was cropped
starting to process MS2685-1-1-1_0066.tif
MS2685-1-1-1_0066.tif was cropped
starting to process MS2685-1-1-1_0067.tif
MS2685-1-1-1_0067.tif was cropped
starting to process MS2685-1-1-1_0068.tif
MS2685-1-1-1_0068.tif was cropped
starting to process MS2685-1-1-1_0069.tif
MS2685-

UnboundLocalError: local variable 'rect' referenced before assignment

In [61]:
count

83

In [62]:
image_paths_list[count]

WindowsPath('data/ex1/MS2685-1-2-2_0015.tif')

In [63]:
# process every image in image_paths_list
print(f'--- starting with image_paths_list[{count}]')

for image_path in image_paths_list[count:count+2]:
    print(f'starting to process {image_path.name}')
    autocrop(image_path, dpi=410, resize_height=1000, padding=50, original_icc_profile=True)
    cropped_image_path = autocrop_directory_path.joinpath('00_cropped', image_path.name)
    if cropped_image_path.is_file():
        print(f'{image_path.name} was cropped')
    count += 1

--- starting with image_paths_list[83]
starting to process MS2685-1-2-2_0015.tif
MS2685-1-2-2_0015.tif was cropped
starting to process MS2685-1-2-2_0016.tif
MS2685-1-2-2_0016.tif was cropped


In [64]:
# process every image in image_paths_list, adding in a set number for count because i done goofed and needed to go
#back to that file

print(f'--- starting with image_paths_list[{count}]')
for image_path in image_paths_list[count:]:
    print(f'starting to process {image_path.name}')
    autocrop(image_path, dpi=410, resize_height=500, padding=50, original_icc_profile=True)
    cropped_image_path = autocrop_directory_path.joinpath('00_cropped', image_path.name)
    if cropped_image_path.is_file():
        print(f'{image_path.name} was cropped')
    count += 1

--- starting with image_paths_list[85]
starting to process MS2685-1-2-2_0017.tif
MS2685-1-2-2_0017.tif was cropped
starting to process MS2685-1-2-2_0018.tif


UnboundLocalError: local variable 'rect' referenced before assignment

In [65]:
count

86

In [66]:
image_paths_list[count]

WindowsPath('data/ex1/MS2685-1-2-2_0018.tif')

In [67]:
# process every image in image_paths_list
print(f'--- starting with image_paths_list[{count}]')

for image_path in image_paths_list[count:count+1]:
    print(f'starting to process {image_path.name}')
    autocrop(image_path, dpi=410, resize_height=1000, padding=50, original_icc_profile=True)
    cropped_image_path = autocrop_directory_path.joinpath('00_cropped', image_path.name)
    if cropped_image_path.is_file():
        print(f'{image_path.name} was cropped')
    count += 1

--- starting with image_paths_list[86]
starting to process MS2685-1-2-2_0018.tif
MS2685-1-2-2_0018.tif was cropped


In [68]:
# process every image in image_paths_list, adding in a set number for count because i done goofed and needed to go
#back to that file

print(f'--- starting with image_paths_list[{count}]')
for image_path in image_paths_list[count:]:
    print(f'starting to process {image_path.name}')
    autocrop(image_path, dpi=410, resize_height=500, padding=50, original_icc_profile=True)
    cropped_image_path = autocrop_directory_path.joinpath('00_cropped', image_path.name)
    if cropped_image_path.is_file():
        print(f'{image_path.name} was cropped')
    count += 1

--- starting with image_paths_list[87]
starting to process MS2685-1-3-3_0001.tif
MS2685-1-3-3_0001.tif was cropped
starting to process MS2685-1-3-3_0002.tif
MS2685-1-3-3_0002.tif was cropped
starting to process MS2685-1-3-3_0003.tif
MS2685-1-3-3_0003.tif was cropped
starting to process MS2685-1-3-3_0004.tif
MS2685-1-3-3_0004.tif was cropped
starting to process MS2686-1-1-2_0001.tif
MS2686-1-1-2_0001.tif was cropped
starting to process MS2686-1-1-2_0002.tif
MS2686-1-1-2_0002.tif was cropped
starting to process MS2686-1-1-2_0003.tif


UnboundLocalError: local variable 'rect' referenced before assignment

In [69]:
count

93

In [71]:
image_paths_list[count]

WindowsPath('data/ex1/MS2686-1-1-2_0003.tif')

In [75]:
# process every image in image_paths_list, adding in a set number for count because i done goofed and needed to go
#back to that file
count = 93
print(f'--- starting with image_paths_list[{count}]')
for image_path in image_paths_list[count:count+2]:
    print(f'starting to process {image_path.name}')
    autocrop(image_path, dpi=410, resize_height=1000, padding=50, original_icc_profile=True)
    cropped_image_path = autocrop_directory_path.joinpath('00_cropped', image_path.name)
    if cropped_image_path.is_file():
        print(f'{image_path.name} was cropped')
    count += 1

--- starting with image_paths_list[93]
starting to process MS2686-1-1-2_0003.tif
MS2686-1-1-2_0003.tif was cropped
starting to process MS2686-1-1-2_0004.tif
MS2686-1-1-2_0004.tif was cropped


In [76]:
# process every image in image_paths_list, adding in a set number for count because i done goofed and needed to go
#back to that file

print(f'--- starting with image_paths_list[{count}]')
for image_path in image_paths_list[count:]:
    print(f'starting to process {image_path.name}')
    autocrop(image_path, dpi=410, resize_height=500, padding=50, original_icc_profile=True)
    cropped_image_path = autocrop_directory_path.joinpath('00_cropped', image_path.name)
    if cropped_image_path.is_file():
        print(f'{image_path.name} was cropped')
    count += 1

--- starting with image_paths_list[95]
starting to process MS2686-1-1-2_0005.tif
MS2686-1-1-2_0005.tif was cropped
starting to process MS2686-1-1-2_0006.tif
MS2686-1-1-2_0006.tif was cropped
starting to process MS2686-1-1-2_0007.tif
MS2686-1-1-2_0007.tif was cropped
starting to process MS2686-1-1-2_0008.tif
MS2686-1-1-2_0008.tif was cropped
starting to process MS2686-1-1-2_0009.tif
MS2686-1-1-2_0009.tif was cropped
starting to process MS2686-1-1-2_0010.tif
MS2686-1-1-2_0010.tif was cropped
starting to process MS2686-1-1-2_0011.tif
MS2686-1-1-2_0011.tif was cropped
starting to process MS2686-1-1-2_0012.tif
MS2686-1-1-2_0012.tif was cropped
starting to process MS2686-1-1-2_0013.tif
MS2686-1-1-2_0013.tif was cropped
starting to process MS2686-1-1-2_0014.tif
MS2686-1-1-2_0014.tif was cropped
starting to process MS2686-1-1-2_0015.tif
MS2686-1-1-2_0015.tif was cropped
starting to process MS2686-1-1-2_0016.tif
MS2686-1-1-2_0016.tif was cropped
starting to process MS2686-1-1-2_0017.tif
MS2686-

UnboundLocalError: local variable 'rect' referenced before assignment

In [77]:
count

115

In [78]:
image_paths_list[count]

WindowsPath('data/ex1/MS2686-1-1-2_0025.tif')

In [79]:
# process every image in image_paths_list, adding in a set number for count because i done goofed and needed to go
#back to that file

print(f'--- starting with image_paths_list[{count}]')
for image_path in image_paths_list[count:count+2]:
    print(f'starting to process {image_path.name}')
    autocrop(image_path, dpi=410, resize_height=1000, padding=50, original_icc_profile=True)
    cropped_image_path = autocrop_directory_path.joinpath('00_cropped', image_path.name)
    if cropped_image_path.is_file():
        print(f'{image_path.name} was cropped')
    count += 1

--- starting with image_paths_list[115]
starting to process MS2686-1-1-2_0025.tif
MS2686-1-1-2_0025.tif was cropped
starting to process MS2686-1-1-2_0026.tif
MS2686-1-1-2_0026.tif was cropped


In [80]:
# process every image in image_paths_list, adding in a set number for count because i done goofed and needed to go
#back to that file

print(f'--- starting with image_paths_list[{count}]')
for image_path in image_paths_list[count:]:
    print(f'starting to process {image_path.name}')
    autocrop(image_path, dpi=410, resize_height=500, padding=50, original_icc_profile=True)
    cropped_image_path = autocrop_directory_path.joinpath('00_cropped', image_path.name)
    if cropped_image_path.is_file():
        print(f'{image_path.name} was cropped')
    count += 1

--- starting with image_paths_list[117]
starting to process MS2686-1-1-3_0001.tif
MS2686-1-1-3_0001.tif was cropped
starting to process MS2686-1-1-3_0002.tif
MS2686-1-1-3_0002.tif was cropped
starting to process MS2686-1-1-3_0003.tif


UnboundLocalError: local variable 'rect' referenced before assignment

In [81]:
count

119

In [82]:
image_paths_list[count]

WindowsPath('data/ex1/MS2686-1-1-3_0003.tif')

In [83]:
# process every image in image_paths_list, adding in a set number for count because i done goofed and needed to go
#back to that file

print(f'--- starting with image_paths_list[{count}]')
for image_path in image_paths_list[count:count+2]:
    print(f'starting to process {image_path.name}')
    autocrop(image_path, dpi=410, resize_height=1000, padding=50, original_icc_profile=True)
    cropped_image_path = autocrop_directory_path.joinpath('00_cropped', image_path.name)
    if cropped_image_path.is_file():
        print(f'{image_path.name} was cropped')
    count += 1

--- starting with image_paths_list[119]
starting to process MS2686-1-1-3_0003.tif
MS2686-1-1-3_0003.tif was cropped
starting to process MS2686-1-1-3_0004.tif
MS2686-1-1-3_0004.tif was cropped


In [84]:
# process every image in image_paths_list, adding in a set number for count because i done goofed and needed to go
#back to that file

print(f'--- starting with image_paths_list[{count}]')
for image_path in image_paths_list[count:]:
    print(f'starting to process {image_path.name}')
    autocrop(image_path, dpi=410, resize_height=500, padding=50, original_icc_profile=True)
    cropped_image_path = autocrop_directory_path.joinpath('00_cropped', image_path.name)
    if cropped_image_path.is_file():
        print(f'{image_path.name} was cropped')
    count += 1

--- starting with image_paths_list[121]
starting to process MS2686-1-1-3_0005.tif
MS2686-1-1-3_0005.tif was cropped
starting to process MS2686-1-1-3_0006.tif
MS2686-1-1-3_0006.tif was cropped
starting to process MS2686-1-1-3_0007.tif


UnboundLocalError: local variable 'rect' referenced before assignment

In [85]:
# process every image in image_paths_list, adding in a set number for count because i done goofed and needed to go
#back to that file

print(f'--- starting with image_paths_list[{count}]')
for image_path in image_paths_list[count:]:
    print(f'starting to process {image_path.name}')
    autocrop(image_path, dpi=410, resize_height=500, padding=50, original_icc_profile=True)
    cropped_image_path = autocrop_directory_path.joinpath('00_cropped', image_path.name)
    if cropped_image_path.is_file():
        print(f'{image_path.name} was cropped')
    count += 1

--- starting with image_paths_list[123]
starting to process MS2686-1-1-3_0007.tif


UnboundLocalError: local variable 'rect' referenced before assignment

In [86]:
# process every image in image_paths_list, adding in a set number for count because i done goofed and needed to go
#back to that file

print(f'--- starting with image_paths_list[{count}]')
for image_path in image_paths_list[count:]:
    print(f'starting to process {image_path.name}')
    autocrop(image_path, dpi=410, resize_height=1000, padding=50, original_icc_profile=True)
    cropped_image_path = autocrop_directory_path.joinpath('00_cropped', image_path.name)
    if cropped_image_path.is_file():
        print(f'{image_path.name} was cropped')
    count += 1

--- starting with image_paths_list[123]
starting to process MS2686-1-1-3_0007.tif
MS2686-1-1-3_0007.tif was cropped
starting to process MS2686-1-1-3_0008.tif
MS2686-1-1-3_0008.tif was cropped
